<a href="https://colab.research.google.com/github/ballmdr/Forex-Prediction-Machine_Learning/blob/master/Forex_Prediction_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
#tf.enable_eager_execution()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import pickle

from datetime import datetime
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

In [0]:
filename = '/content/drive/My Drive/export/M30_all_GBPUSD.csv'
df = pd.read_csv(filename, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Datetime'] = pd.to_datetime(df.Date + ' ' + df.Time)
df.set_index('Datetime', inplace=True)

# Features for Momentum Strategy
df['Returns'] = - df.Close.pct_change(-1)

df['MA35'] = df.Close.rolling(35).mean()
df['MA70'] = df.Close.rolling(70).mean()
df['MA200'] = df.Close.rolling(200).mean()

# ma35 > ma70 = bullish
df['35_70'] = np.where(df.MA35 >= df.MA70, 1, 0)

# price > ma200 = bullish
df['bias_trend'] = np.where(df.Close.values >= df.MA200, 1, 0) 

df['body_candles'] = df.Open - df.Close
df['high_low'] = df.High - df.Low

df.dropna(inplace=True)

def norm_features(x):
  return ((x - x.mean()) / (x.max() - x.min())).astype(np.float32)

# normalize
df['Returns'] = norm_features(df.Returns)
df['Volume'] = norm_features(df.Volume)
df['body_candles'] = norm_features(df.body_candles)
df['high_low'] = norm_features(df.high_low)

df['Target'] = np.zeros((len(df)))
df['Target'].loc[df.Returns > df.Returns.quantile(.20)] = 1
df['Target'].loc[df.Returns > df.Returns.quantile(.80)] = 2
#df['Target'].loc[df.Returns > df.Returns.quantile(.60)] = 3
#df['Target'].loc[df.Returns > df.Returns.quantile(.80)] = 4

df['Time_h4'] = np.zeros((len(df)))
df['Time_h4'].loc[df.Time >= '04:00'] = 1
df['Time_h4'].loc[df.Time >= '08:00'] = 2
df['Time_h4'].loc[df.Time >= '12:00'] = 3
df['Time_h4'].loc[df.Time >= '16:00'] = 4
df['Time_h4'].loc[df.Time >= '20:00'] = 5

time_h4 = pd.DataFrame(to_categorical(df.Time_h4, num_classes=6))
time_h4 = time_h4.set_index(df.index)
time_h4.columns = ['00:00', '04:00', '08:00', '12:00', '16:00', '20:00']

df = pd.concat([df, time_h4], axis=1)

df.drop(['Date', 'Time', 'Time_h4', 'Open', 'High', 'Low', 'Close', 'Returns', 'MA35', 'MA70', 'MA200'], axis=1, inplace=True)

X = df.drop(['Target'], axis=1).values
Y = df.Target.values


# convert y into categorical
K = 3

Y = to_categorical(Y, num_classes=K)

# split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

print('train shape: ', x_train.shape)
print('test shape: ', x_test.shape)

print('Features: ', df.columns)

In [0]:
# Features ['Volume', '35_70', 'bias_trend', 'body_candles', 'high_low', 'Target', '00:00', '04:00', '08:00', '12:00', '16:00', '20:00']
# Output 3 Classes

# use keras for benchmark
early_stop = tf.keras.callbacks.EarlyStopping(patience=3, monitor='loss', mode='min')

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(X.shape[1],), activation=tf.nn.tanh),
    tf.keras.layers.Dense(128, activation=tf.nn.tanh),
    tf.keras.layers.Dense(128, activation=tf.nn.tanh),
    tf.keras.layers.Dense(128, activation=tf.nn.tanh),
    tf.keras.layers.Dense(128, activation=tf.nn.tanh),
    tf.keras.layers.Dense(K, activation=tf.nn.softmax)
])

model.compile(loss='categorical_crossentropy', optimizer=tf.train.RMSPropOptimizer(0.000001), metrics=['accuracy'])

init = True
if not init:
  model.load_weights('/content/drive/My Drive/Colab Notebooks/benchmark_weight.h5')
  
epochs = 100
#runned 100

history = model.fit(x_train, y_train, epochs=epochs, callbacks=[early_stop], validation_data=(x_test, y_test), validation_split=0.3, verbose=1, batch_size=32)

key = 'loss'
plt.figure(figsize=(10,5))
val = plt.plot(history.epoch, history.history['val_'+key], '--', label='Test')
plt.plot(history.epoch, history.history[key], color=val[0].get_color(), label='Train')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.xlim([0,max(history.epoch)])
plt.show()

model.save_weights('/content/drive/My Drive/Colab Notebooks/benchmark_weight.h5')